In [24]:
from datasets import load_dataset
import numpy as np
from collections import defaultdict
dataset = load_dataset("sst", "default")

In [25]:
#naive bayes
def train_naive_bayes(D, C):
    N_doc = len(D)
    logprior = {}
    loglikelihood = {}
    V = set()
    bigdoc = defaultdict(list)
    word_counts_per_class = defaultdict(lambda: defaultdict(int))

    # Calculate logprior and collect documents for each class
    for c in C:
        D_c = [doc_tokens for doc_tokens, cl in D if cl == c]
        N_c = len(D_c)
        logprior[c] = np.log(N_c / N_doc)
        for doc in D_c:
            bigdoc[c] += doc
            V.update(doc)
            for word in doc:
                word_counts_per_class[word][c] += 1

    V = list(V)

    # Calculate loglikelihood
    for w in V:
        for c in C:
            count_w_c = word_counts_per_class[w][c]
            loglikelihood[(w, c)] = np.log((count_w_c + 1) / (sum(word_counts_per_class[w].values()) + len(V)))

    return logprior, loglikelihood, V

def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):
    sum_scores = {c: logprior[c] for c in C}
    for word in testdoc:
        if word in V:
            for c in C:
                sum_scores[c] += loglikelihood.get((word, c), 0)
    return max(sum_scores, key=sum_scores.get)

In [26]:
classes = {
    0: "Very Negative",
    1: "Negative",
    2: "Neutral",
    3: "Positive",
    4: "Very Positive",
}
def map_dataset(data):
    if data <= 0.2:
        return 0  # very negative
    elif data <= 0.4:
        return 1  # negative
    elif data <= 0.6:
        return 2  # neutral
    elif data <= 0.8:
        return 3  # positive
    else:
        return 4  # very positive
def data_to_tokens(data):
    documents = []
    for entry in data:
        tokens = entry['tokens'].split("|")
        label = map_dataset(entry['label'])
        documents.append((tokens, label))
    return documents

In [27]:
# Prepare training and testing data
train_documents = data_to_tokens(dataset['train'])
test_documents = data_to_tokens(dataset['test'])

In [28]:
# Train Naive Bayes classifier
logprior, loglikelihood, vocab_list = train_naive_bayes(train_documents, [0, 1, 2, 3, 4])

# Test the classifier on the test dataset
correct_predictions = 0
total_predictions = len(test_documents)

for doc_tokens, actual_class in test_documents:
    predicted_class = test_naive_bayes(doc_tokens, logprior, loglikelihood, [0, 1, 2, 3, 4], vocab_list)
    if predicted_class == actual_class:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)

# Test the classifier on a sample document
random_phrase = "This movie was fantastic! I loved every minute of it."
random_score = np.random.uniform(0, 1)
actual_class = map_dataset(random_score)
predicted_class = test_naive_bayes(random_phrase.split(), logprior, loglikelihood, [0, 1, 2, 3, 4], vocab_list)
print("Random phrase:", random_phrase)
print("Predicted class:", predicted_class)
print("Actual class:", actual_class)

Accuracy: 0.36742081447963804
Random phrase: This movie was fantastic! I loved every minute of it.
Predicted class: 1
Actual class: 0


In [29]:
#comparison with sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Prepare training and testing data
train_texts = [' '.join(tokens) for tokens, _ in train_documents]
test_texts = [' '.join(tokens) for tokens, _ in test_documents]
train_labels = [label for _, label in train_documents]
test_labels = [label for _, label in test_documents]

# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB(alpha=1.0))
])

# Train the classifier
pipeline.fit(train_texts, train_labels)

# Predict on the test set
predicted_labels = pipeline.predict(test_texts)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy using scikit-learn:", accuracy)


Accuracy using scikit-learn: 0.4090497737556561


In [34]:
#logistic regression
# Feature representation
def generate_word_bigrams(sentence):
    words = sentence.split()
    return [(words[i], words[i+1]) for i in range(len(words)-1)]

def generate_features(sentences, ):
    unique_bigrams = set()
    for sentence in sentences:
        bigrams = generate_word_bigrams(sentence)
        unique_bigrams.update(bigrams)
    unique_bigrams = sorted(list(unique_bigrams))
    features = np.zeros((len(sentences), len(unique_bigrams)), dtype=int)
    for i, sentence in enumerate(sentences):
        bigrams = generate_word_bigrams(sentence)
        for j, bigram in enumerate(unique_bigrams):
            if bigram in bigrams:
                features[i, j] = 1
    return features


# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [31]:
# Model Training
def train_logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros((n, 1))
    for i in range(num_iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient
    return theta

In [32]:
# Prediction
def predict(X, theta):
    z = np.dot(X, theta)
    h = sigmoid(z)
    return (h >= 0.5).astype(int)

# Evaluation
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)


In [33]:
# Example usage
sentences = ["I love this movie very much", "This movie is great"]
X_train = generate_features(sentences)
y_train = np.array([1, 0])  # Example labels
y_train = y_train.reshape(-1, 1)  # Reshape y_train to match theta dimensions

# Train logistic regression
learning_rate = 0.01
num_iterations = 1000
theta = train_logistic_regression(X_train, y_train, learning_rate, num_iterations)

# Example prediction
X_test = generate_features(["I hate this movie"])
y_pred = predict(X_test, theta)
print("Predicted label:", y_pred[0])  # Output: 0 (negative)

# Example evaluation
accuracy_score = accuracy(y_train, predict(X_train, theta))
print("Training accuracy:", accuracy_score)

ValueError: shapes (1,3) and (8,1) not aligned: 3 (dim 1) != 8 (dim 0)